In [ ]:
# OS library
import sys
sys.path.append('../')

import glob
import pandas as pd
import numpy as np

from urllib.request import urlopen
import json

# Magic to reload notebook
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = 100

# Covid Saúde

Painel de casos de doença pelo coronavírus 2019 (COVID-19) no Brasil pelo Ministério da Saúde ([website](https://covid.saude.gov.br/))

In [ ]:
df_covid_saude = pd.read_parquet("../data/processed/covid_saude.parquet")
df_acoes = pd.read_csv('../data/raw/boletim_covid/acoes.csv', sep='\t', encoding='latin-1')
df_referencias = pd.read_csv('../data/raw/boletim_covid/referencias.csv', sep='\t', encoding='latin-1')

In [ ]:
df_covid_saude_mun = df_covid_saude.loc[df_covid_saude['municipio'].notna()]

df_covid_saude_mun = df_covid_saude_mun[[
    'regiao', 'estado', 'municipio', 'codmun', 'nomeRegiaoSaude',
    'codRegiaoSaude', 'data', 'obitosNovos'
]]

df_covid_saude_mun['data'] = pd.to_datetime(df_covid_saude_mun['data'])

df_depara_wn_day = df_covid_saude_mun[['data']].drop_duplicates()
df_depara_wn_day['weekday'] = df_depara_wn_day['data'].dt.weekday
df_depara_wn_day['week_number'] = 0
df_depara_wn_day.loc[df_depara_wn_day['weekday'] == 0, 'week_number'] = 1
df_depara_wn_day['gropuby_col'] = 1
df_depara_wn_day['week_number'] = df_depara_wn_day.groupby(
    'gropuby_col')['week_number'].cumsum()
df_covid_saude_mun = df_covid_saude_mun.merge(
    df_depara_wn_day[['data', 'week_number']], how='left', on=['data'])

df_covid_saude_mun.sort_values(by=['municipio', 'data'],
                               ascending=[True, True],
                               inplace=True)
df_covid_saude_br = df_covid_saude_mun[['week_number',
                                        'data']].drop_duplicates(
                                            subset=['week_number'],
                                            keep='first')
df_covid_saude_reg = df_covid_saude_mun[['regiao', 'week_number',
                                         'data']].drop_duplicates(
                                             subset=['regiao', 'week_number'],
                                             keep='first')
df_covid_saude_state = df_covid_saude_mun[[
    'estado', 'week_number', 'data'
]].drop_duplicates(subset=['estado', 'week_number'], keep='first')
df_covid_saude_regsau = df_covid_saude_mun[[
    'nomeRegiaoSaude', 'codRegiaoSaude', 'week_number', 'data'
]].drop_duplicates(subset=['nomeRegiaoSaude', 'codRegiaoSaude', 'week_number'],
                   keep='first')
df_covid_saude_city = df_covid_saude_mun[[
    'municipio', 'codmun', 'week_number', 'data'
]].drop_duplicates(subset=['municipio', 'codmun', 'week_number'], keep='first')

df_weekly_deaths = df_covid_saude_mun.groupby(
    ['week_number'])['obitosNovos'].sum().reset_index(name='new_deaths_week')

df_weekly_deaths_br = df_covid_saude_mun.groupby([
    'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_region = df_covid_saude_mun.groupby([
    'regiao', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_state = df_covid_saude_mun.groupby([
    'estado', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_regsau = df_covid_saude_mun.groupby([
    'nomeRegiaoSaude', 'codRegiaoSaude', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')
df_weekly_deaths_city = df_covid_saude_mun.groupby([
    'municipio', 'codmun', 'week_number'
])['obitosNovos'].sum().reset_index(name='new_deaths_week_division')

df_covid_saude_br = df_covid_saude_br.merge(df_weekly_deaths_br, how='left', on=['week_number']) \
    .merge(df_weekly_deaths, how='left', on=['week_number'])
df_covid_saude_reg = df_covid_saude_reg.merge(df_weekly_deaths_region, how='left', on=['regiao', 'week_number']) \
    .merge(df_weekly_deaths, how='left', on=['week_number'])
df_covid_saude_state = df_covid_saude_state.merge(df_weekly_deaths_state, how='left', on=['estado', 'week_number']) \
    .merge(df_weekly_deaths, how='left', on=['week_number'])
df_covid_saude_regsau = df_covid_saude_regsau.merge(df_weekly_deaths_regsau, how='left', on=['nomeRegiaoSaude', 'codRegiaoSaude', 'week_number']) \
    .merge(df_weekly_deaths, how='left', on=['week_number'])
df_covid_saude_city = df_covid_saude_city.merge(df_weekly_deaths_city, how='left', on=['municipio', 'codmun', 'week_number']) \
    .merge(df_weekly_deaths, how='left', on=['week_number'])

df_covid_saude_br['percentage_deaths'] = df_covid_saude_br[
    'new_deaths_week_division'] / df_covid_saude_br['new_deaths_week']
df_covid_saude_reg['percentage_deaths'] = df_covid_saude_reg[
    'new_deaths_week_division'] / df_covid_saude_reg['new_deaths_week']
df_covid_saude_state['percentage_deaths'] = df_covid_saude_state[
    'new_deaths_week_division'] / df_covid_saude_state['new_deaths_week']
df_covid_saude_regsau['percentage_deaths'] = df_covid_saude_regsau[
    'new_deaths_week_division'] / df_covid_saude_regsau['new_deaths_week']
df_covid_saude_city['percentage_deaths'] = df_covid_saude_city[
    'new_deaths_week_division'] / df_covid_saude_city['new_deaths_week']

df_covid_saude_br.fillna(0, inplace=True)
df_covid_saude_reg.fillna(0, inplace=True)
df_covid_saude_state.fillna(0, inplace=True)
df_covid_saude_regsau.fillna(0, inplace=True)
df_covid_saude_city.fillna(0, inplace=True)

df_covid_saude_grouped = pd.concat([
    df_covid_saude_city, df_covid_saude_regsau, df_covid_saude_state,
    df_covid_saude_reg, df_covid_saude_br
])

##################################################################
# Tratamento de dados das noticias
##################################################################

df_acoes['texto'] = df_acoes['texto'].str.replace('\x93', '"')\
                                     .str.replace('\x94', '"')\
                                     .str.replace('\x91', "'")\
                                     .str.replace('\x92', "'")\
                                     .str.replace('\x85', '...')

df_acoes['data'] = pd.to_datetime(df_acoes['data'], format='%d/%m/%y')

df_acoes['texto'] = df_acoes['texto'].str.wrap(30)
df_acoes['texto'] = df_acoes['texto'].apply(lambda x: x.replace('\n', '<br>'))
df_acoes = df_acoes.merge(df_depara_wn_day, how='left', on='data')

df_acoes['noticia'] = '<b>Data: </b>' + df_acoes['data'].dt.strftime(
    '%d/%m/%Y') + '<br><b>Tipo: </b>' + df_acoes['tipo'].astype(
        str) + '<br><b>Notícia: </b>' + df_acoes['texto']

df_noticias = df_acoes.groupby([
    'week_number'
])['noticia'].apply(lambda x: '<br><br>'.join(x)).reset_index()

# df_noticias['noticia'] = '<div style="height:120px;width:120px;border:1px solid #ccc;font:16px/26px Georgia, Garamond, Serif;overflow:auto;">' + df_noticias['noticia'] + '</div>'
# df_noticias['noticia'] = '<p>' + df_noticias['noticia'] + '</p>'
df_covid_saude_grouped = df_covid_saude_grouped.merge(df_noticias,
                                                      how='left',
                                                      on='week_number')

df_covid_saude_grouped['noticia'] = df_covid_saude_grouped['noticia'].where(pd.notnull(df_covid_saude_grouped['noticia']), None)

In [ ]:
# df_covid_saude_mun.to_parquet('../data/app/covid_saude_obito_municipio.parquet', index=False)
df_covid_saude_grouped.to_parquet('../data/app/covid_saude_obito_grouped.parquet', index=False)

In [ ]:
df_teste = pd.read_parquet('../data/app/est_cidade.parquet')

In [ ]:
df_teste.drop_duplicates().dropna().to_parquet('../data/app/est_cidade.parquet', index=False)